## Food2Fork API ##
www.food2fork.com/about/api

In [20]:
import requests
import urllib2
import json

#### Load API credentials ####

In [25]:
def loadCredentials():
    filename = 'secrets.txt'
    for line in open(filename).readlines():
        if "API" in line:    
            api_key = line.split(": ")[1].translate(None,'\n')
        
    return api_key

In [48]:
API_KEY = loadCredentials()

In [49]:
URL_API = 'http://food2fork.com/api'
URL_SEARCH = URL_API + '/search/?'
URL_GET = URL_API + '/get/?'

# Base URLs
search_url = "http://food2fork.com/api/search"
recipe_url = "http://food2fork.com/api/get"

#### Make a GET request to the API ####

In [50]:
# api_request = search_url + ""
api_request = "http://food2fork.com/api/search?key="+API_KEY+"&q=shredded%20chicken"

# I don't understand headers well enough yet, but I got a 403 error without this header
# https://en.wikipedia.org/wiki/User_agent
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}

request = urllib2.Request(api_request, headers=hdr)
response = urllib2.urlopen(request)
raw = response.read()
json_obj = json.loads(raw)
json_obj.viewkeys()


dict_keys([u'count', u'recipes'])

In [47]:
[recipe['title'] for recipe in json_obj['recipes']]

[u'Buffalo Chicken Grilled Cheese Sandwich',
 u'Slow Cooker Chicken Tortilla Soup',
 u'Mac and Cheese with Roasted Chicken, Goat Cheese, and Rosemary',
 u'White Chicken Enchiladas',
 u'Cauliflower Pizza Crust (with BBQ Chicken Pizza)',
 u'Bacon Wrapped Buffalo Chicken Jalapeno Poppers',
 u'Buffalo Chicken Potato Skins',
 u'Sweet Chili Chicken Quesadilla',
 u'BBQ Chicken Quinoa Salad',
 u'Lemon Chicken Orzo Soup',
 u'Barbecue Chicken Pizza',
 u'Buffalo Chicken Quinoa Fritters',
 u'Spicy Peanut Chicken Grilled Cheese Sandwich (aka The Chicken Satay Melt)',
 u'Cheesy Grown Up SpaghettiOs',
 u'Slow Cooker Chicken Enchilada Soup',
 u'Tortilla Soup',
 u'Chicken and Tortellini Soup',
 u'Chicken Enchilada Stuffed Zucchini',
 u'Chicken, Spinach, and Noodle Casserole',
 u'Easy Chicken and Rice Soup',
 u'Chicken, Roasted Red Pepper and Goat Cheese Lasagna',
 u'Buffalo Chicken Sushi',
 u'Cauliflower Pizza Crust (with BBQ Chicken Pizza)',
 u'Chicken Bacon and Ranch Stacked Enchiladas',
 u'Bang Bang

In [54]:
json_obj['recipes'][0]

{u'f2f_url': u'http://food2fork.com/view/35171',
 u'image_url': u'http://static.food2fork.com/Buffalo2BChicken2BGrilled2BCheese2BSandwich2B5002B4983f2702fe4.jpg',
 u'publisher': u'Closet Cooking',
 u'publisher_url': u'http://closetcooking.com',
 u'recipe_id': u'35171',
 u'social_rank': 100.0,
 u'source_url': u'http://www.closetcooking.com/2011/08/buffalo-chicken-grilled-cheese-sandwich.html',
 u'title': u'Buffalo Chicken Grilled Cheese Sandwich'}